<h2>OBSERVATIONAL ASTROPHYSICS – FALL 2019 Reduction Exercise: Part 2</h2>

In Part 1 you inspected your images.  Some of you made a master bias frame.  We have since found out that the bias frames contain residual structure that we don't understand and we will therefore only subtract the darks, which contain the bias already.

In Part 2 you will will:

1. combine your dark frames (which include their own bias);
4. subtract the dark frame from each of your science and flat field images;
5. make a combined flatfield image in each band;
6. flatfield the individual flatfield images (as a check) and the science images;

Remember that your raw data are stored in ~/RFSLAB/USER_DPT/_PUBLIC/ASTR596/Data/Raw/< your night>
and your reduced images are stored in ~/RFSLAB/USER_DPT/< KUID>/ASTR596/Data/Reduced/< your_night> 

In [1]:
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc
from matplotlib import pyplot as plt
from ccdproc import Combiner
from astropy.io import ascii
from astropy.io import fits
import os

<h4>Combine the darks</h4>

We will combine the darks in the same way that we combined the bias frames in part1.  This is a section of code that you will need to run and do some modifications on.

In [2]:
darklist = []
darklist_180s_good = np.genfromtxt("C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/darklist_180s_good.txt",dtype=None,encoding=None)
path = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/darklist"
for x in os.listdir(path):
    
    if (x in darklist_180s_good) == True:
    
        #read that into a CCDData object.  This allows you to specify a unit, which we indicate as "adu"
        imstr = path + "/" + x
        im = CCDData.read(imstr,unit = "adu")
        fitsheader = fits.getheader(imstr)
        #make a list of all the CCDData instances of each image
        darklist = darklist + [im]
    else: 
        pass 

darklist_comb = Combiner(darklist)

#now create masks with sigma clipping algorithm
#This creates a mask for each image that is the size of each image but which has 0 values
#everywhere except those pixels in excess of low_thresh and high_thresh sigma from the median.
#Those pixels get a value of 1
#*************
#given the number of pixels in your image how must you set high_thresh to avoid flagging more than
#one pixel because of expected statistical variations.  Assume the noise is distributed like a Gaussian.
darklist_comb.sigma_clipping(low_thresh=2, high_thresh=6 , func=np.ma.median)

#Use these masks to combine the images.  This now becomes your master bias
master_bias = darklist_comb.average_combine()

#now write out the master bias
redpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/red"
masterbiaspath = redpath + '/master_dark.fit'
master_bias.write(masterbiaspath,overwrite=True)

This defines a routine that subtracts the dark frame from a list of images.  There is one command that I want you to modify after you read the manual for subtracting darks at https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html

In [5]:
def dark_sub(rawpath, redpath, master_darkfile):
    #go through each image in a list, subtract the master dark from that image, 
    #and write the image out again.
    #imlist_name: the name of the file (without path) that contains the list of 
    #images to subtract the dark from
    #rawpath: the path with the raw data
    #redpath: the path of directory with your reduced data
    #filepath: the path of the directory with the lists of files
    #dark: the filename of the master dark

    masterdarkpath = redpath + master_darkfile
    master_dark = CCDData.read(masterdarkpath,unit = "adu")
   
    for x in os.listdir(rawpath):
    
            impath = rawpath + x
            #load file into CCDData objet
            im = CCDData.read(impath,unit = "adu")
            #retrieve the fits header for each image
            fitsheader = fits.getheader(impath)
            print(fitsheader)
            #convert EXPTIME to sec, as microsec is not supported by astropy.units
            fitsheader['EXPTIME'] = fitsheader['EXPTIME'] / 1.e6

            #***********  
            #complete the appropriate expression from https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html
            #subtract dark frame
            
            im_dsub = ccdproc.subtract_dark(im,master_dark,exposure_time='EXPTIME', exposure_unit=u.second,
                                            scale=False)

            #this attaches the fits header back to the ccddate object that will be written
            im_dsub.header = fitsheader
            #now write out the file
            dsub_imname = imname.replace('.fit','d.fit',1)
            #print(bsub_imname)
            dsub_path = redpath + dsub_imname
            im_dsub.write(dsub_path,overwrite=True)
            line = fp.readline()



<h4> subtract the darks </h4>

This section of code is where you have to call the `dark_sub` function for each of your list of images.  The output will be a set of images in your `Reduced` directory that have the `d` suffix and have had the dark current subtracted.  

The dark subtraction we implement on our flatfields is not correct because we don't have dark frames with the exposure time of our flat fields.  Subtracting a 180s exposure time dark frame is wrong but it's the best we can do.

In [6]:

#**************
#put in the appropriate file name for each of the flats and science frames
#subtract bias from flats in each band (you will need two calls to this function)
redpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/red/"

master_darkfile = 'master_dark.fit'
#subtract master dark from darks.  This just just a check to see how well the dark 
#current is subtracted from each dark frame.
#DARKs rawpath 
rawpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/darklist/"
dark_sub(rawpath, redpath, master_darkfile)

#subtract master dark from flats.  
#FLATS R rawpath
rawpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/flats_r/"
dark_sub(rawpath, redpath, master_darkfile)
#FLATS G rawpath
rawpath = rawpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/flats_g/"
dark_sub(rawpath, redpath, master_darkfile)

#subtract master dark from science frames
rawpath = rawpath = "C:/Users/19133/Documents/Obs/My repository/ObservingProject_Student/files/science/"
dark_sub(rawpath, redpath, master_darkfile)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
SIMPLE  =                    T / C# FITS: 10/20/2021 12:03:57 AM                BITPIX  =                   16                                                  NAXIS   =                    2 / Dimensionality                                 NAXIS1  =                 2048                                                  NAXIS2  =                 2048                                                  SENSTEMP= '-274.0  '           / Sensor Temperature (C)                         CCD-TEMP=                  -15 / CCD Temperature (C)                            YORGSUBF=                    0 / Subframe origin on Y axis                      XORGSUBF=                    0 / Subframe origin on X axis                      YBINNING=                    2 / Binning factor used on the Y axis              XBINNING=                    2 / Binning factor used on the X axis              EX

KeyError: "Keyword 'EXPTIME' not found."

Now combine the flatfields into "master flats".  In this process you will scale each object to their mean before combining.  

Here we will use a median combination instead of a sigma clipping combination.  In the cell below please describe why this is a good idea.

<h4>Your answer here:</h4>

Towards the bottom of this routine we will be using a "Lambda" function to normalize our images.  These are small anonymous throw-away functions that are only used once in a code.  Their use is described here https://www.python-course.eu/python3_lambda.php.  

In [ ]:
def master_flatmake(imlist_name, redpath, filepath, master_flatname):
    #a function that makes a master flatfield, taking its flats from a list.  
    #The process will be very similar to that for bias frames, though we 
    #1) normalize the frames before combining and 
    #2) make it a function as we may need to generate flats for mutliple bands

    #imlist_name: the name of the file (without path) that contains the list of images that will be combined
    #filepath: the path of the directory with the lists of files
    #redpath: the path of directory with your reduced data
    #master_flatname: the name of the combined master flatfield

    imlistpath = filepath + imlist_name


    #initialize list of flat frames
    flat_imlist = []
    #this way of opening the file ensures that it closes after the loop is done.
    with open(imlistpath,'r') as fp:
        #read first line
        line = fp.readline()

        #build the combine list of all images
        #read every subsequent line
        while line:
            #this removes the trailing newline charactter and converts the output list to a scalar
            imname = line.split()
            imname = imname[0]
        
            #create the image name, including the path
            imstr = redpath + imname
            #read that into a CCDData object.  This allows you to specify a unit
            im = CCDData.read(imstr)
            #get the header for each image.  This will just get attached to the final combined image
            fitsheader = fits.getheader(imstr)

            #make a list of all the CCDData instances of each image
            flat_imlist = flat_imlist + [im]

            line = fp.readline()

    #Combiner list of all flat images
    flat_combiner = Combiner(flat_imlist)
    
    #######################################
    #this is an anonymous throw away function that takes an argument and is intended
    #to be used just once.  In this case, the scaling functionality of the combiner is 
    #set to scale by the inverse of the mean, so images with high counts get scaled down and
    #images wtih low counts get scaled up
    scaling_func = lambda arr: 1/np.ma.average(arr)
    flat_combiner.scaling = scaling_func

    #combine the scaled images
    #****************
    #insert appropriate expression from https://ccdproc.readthedocs.io/en/latest/image_combination.html for 
    #a median combination
    master_flat = 

    #this attaches the fits header back to the ccddate object that will be written
    master_flat.header = fitsheader
    

    #now write out the master flat
    masterflatpath = redpath + master_flatname
    master_flat.write(masterflatpath,overwrite=True)


Now use this function to generate master flats.

Before you do this, you will need to make files containing a list of all your dark subtracted skyflats.  With one list for each band.  You can call it flatlist_\<filter\>_dsub

In [ ]:
#***********
#make the appropriate calles to master_flatmake to make the flatfields.
#You will need to provide the appropriate filename
master_flatmake('', redpath, filepath, 'master_flat_g.fits')
master_flatmake('', redpath, filepath, 'master_flat_r.fits')

Write in the cell below how you can assess whether or not you have a good master flatfield.  What characteristics might you look for and does your master flatfield image have them.

<h4>Your answer goes here:</h4>

-------------------------------
Now we will flatfield all of our science images.  This is similar to the process for bias subtraction except that we will be dividing by the master flat, as opposed to subtracting the master bias.

In [ ]:
def flat_corr(imlist_name, redpath, filepath, masterflat_name):
    #go through each image in a list, subtract the master bias from that image, and write the image out again.
    #imlist_name: the name of the file (without path) that contains the list of images to subtract the bias from
    #redpath: the path of directory with your reduced data
    #master_name: the filename of the master flatfield

    masterflat_path = redpath + masterflat_name
    master_flat = CCDData.read(masterflat_path)
    
    imlistpath= filepath + imlist_name
    with open(imlistpath,'r') as fp:
        #read first line
        line = fp.readline()

        #read every subsequent line
        while line:
            #read in each file
            imname = line.split()
            imname = imname[0]
            impath = redpath + imname
            #load file into CCDData objet
            im = CCDData.read(impath)
            
            #get the header for each image.  This will just get attached to the final combined image
            fitsheader = fits.getheader(imstr)
            
            #***********  
            #insert appropriate expression from https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html
            #subtract bias frame
            im_flat = 

            #this attaches the fits header back to the ccddate object that will be written
            im_flat.header = fitsheader
            #now write out the file
            flat_imname = imname.replace('.fit','f.fit',1)

            imflat_path = redpath + flat_imname
            im_flat.write(imflat_path,overwrite=True)
            line = fp.readline()



In [ ]:
#flatfield correct all the flats
flat_corr('', redpath, filepath, 'master_flat_g.fits')
flat_corr('', redpath, filepath, 'master_flat_r.fits')

#subtract bias from science frames
flat_corr('',redpath, filepath, 'master_flat_g.fits')
flat_corr('',redpath, filepath, 'master_flat_r.fits')

In the cell below, I would like you comment on whether you think the flatfielded science images is good or not.  Try to be quantitative in your esimate.  You will need to look at the images in ds9.  *You can ignore the vignetting by the guider in this assessment.*

<h4>Your answer goes here:</h4>